# Introduction to dplyr and tidyr (part 2)

In [2]:
library(dplyr)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [3]:
surveys <- read.csv('https://github.com/WSU-DataScience/DSCI_210_R_notebooks/raw/main/portal_data_joined.csv')
head(surveys)

record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight,genus,species,taxa,plot_type
1,7,16,1977,2,NL,M,32,NA,Neotoma,albigula,Rodent,Control
72,8,19,1977,2,NL,M,31,NA,Neotoma,albigula,Rodent,Control
224,9,13,1977,2,NL,,NA,NA,Neotoma,albigula,Rodent,Control
266,10,16,1977,2,NL,,NA,NA,Neotoma,albigula,Rodent,Control
349,11,12,1977,2,NL,,NA,NA,Neotoma,albigula,Rodent,Control
363,11,12,1977,2,NL,,NA,NA,Neotoma,albigula,Rodent,Control


# Outline

* Grouped aggregations
* Joins
* Gather/spread (Stack/Unstack)
* The JMP --> `dplyr/tidyr` translation

# Aggregation

### Simple aggregation 

In [4]:
head(surveys)
surveys %>%
  summarize(avg_wgt = mean(weight))

record_id,month,day,year,plot_id,species_id,sex,hindfoot_length,weight,genus,species,taxa,plot_type
1,7,16,1977,2,NL,M,32,NA,Neotoma,albigula,Rodent,Control
72,8,19,1977,2,NL,M,31,NA,Neotoma,albigula,Rodent,Control
224,9,13,1977,2,NL,,NA,NA,Neotoma,albigula,Rodent,Control
266,10,16,1977,2,NL,,NA,NA,Neotoma,albigula,Rodent,Control
349,11,12,1977,2,NL,,NA,NA,Neotoma,albigula,Rodent,Control
363,11,12,1977,2,NL,,NA,NA,Neotoma,albigula,Rodent,Control


avg_wgt
NA


In [5]:
surveys %>%
  filter(complete.cases(weight)) %>%
  summarize(avg_wgt = mean(weight))

avg_wgt
42.67243


In [6]:
surveys %>%
  summarize(avg_wgt = mean(weight, na.rm=TRUE))

avg_wgt
42.67243


### Grouped aggregation

***One grouping variable:***

In [7]:
surveys %>%
    group_by(taxa) %>%
    summarize(avg_wt = mean(weight, na.rm = TRUE))

`summarise()` ungrouping output (override with `.groups` argument)


taxa,avg_wt
Bird,NaN
Rabbit,NaN
Reptile,NaN
Rodent,42.67243


***Two grouping variables:***

In [8]:
surveys %>%
  group_by(taxa, sex) %>%
  summarize(cnt = n())

`summarise()` regrouping output by 'taxa' (override with `.groups` argument)


taxa,sex,cnt
Bird,,450
Rabbit,,75
Reptile,,14
Rodent,,1209
Rodent,F,15690
Rodent,M,17348


***A mutate-group-aggregate with  the ever-useful `ifelse()`***

In [9]:
surveys %>%
  mutate(rodent = ifelse(taxa=='Rodent','yes','no')) %>% 
  group_by(rodent, sex) %>%
  summarize(cnt = n(), avg_wt = mean(weight,na.rm=TRUE))


`summarise()` regrouping output by 'rodent' (override with `.groups` argument)


rodent,sex,cnt,avg_wt
no,,539,NaN
yes,,1209,64.74257
yes,F,15690,42.17055
yes,M,17348,42.99538


***A mutate-group-aggregate-filter (note "" != NA):***

In [10]:
surveys %>%
  mutate(rodent = ifelse(taxa=='Rodent','yes','no')) %>% 
  group_by(rodent, sex) %>%
  summarize(cnt = n(), avg_wt = mean(weight,na.rm=TRUE)) %>%
  filter(sex!="")

`summarise()` regrouping output by 'rodent' (override with `.groups` argument)


rodent,sex,cnt,avg_wt
yes,F,15690,42.17055
yes,M,17348,42.99538


# Stack and Unstack


* `library(tidyr)`
* Stack $\rightarrow$ `gather`
* Unstack $\rightarrow$ `spread` 

### `gather()`

In [11]:
library('tidyr')

##### Imperative:
```{R}
gather(data, key = , value = , col1, col2, col3, ...)
```

#### Piping:
```{R}
data %>% 
  gather(key = , value = , col1, col2, col3, ...)
```

Arguments are:

1. label column name 
2. data column name
3. then a list of columns to stack

A familiar example

In [12]:
sales <- read.csv("https://github.com/WSU-DataScience/DSCI_210_R_notebooks/raw/main/auto_sales.csv")
sales

Salesperson,Compact,Sedan,SUV,Truck
Ann,22,18,15,12
Bob,19,12,17,20
Yolanda,19,8,32,15
Xerxes,12,23,18,9


#### Option 1: spell out all stacking columns

In [13]:
library(tidyr)
stacked_sales <-
  sales %>%
  gather(key = auto_type,value = num_sales, Compact, Sedan, SUV, Truck)
head(stacked_sales)

Salesperson,auto_type,num_sales
Ann,Compact,22
Bob,Compact,19
Yolanda,Compact,19
Xerxes,Compact,12
Ann,Sedan,18
Bob,Sedan,12


#### Option 2: refer to column range

In [14]:
stacked_sales <-
  sales %>%
  gather(key = auto_type,value = num_sales,Compact:Truck)
head(stacked_sales)

Salesperson,auto_type,num_sales
Ann,Compact,22
Bob,Compact,19
Yolanda,Compact,19
Xerxes,Compact,12
Ann,Sedan,18
Bob,Sedan,12


#### Option 3: select by exclusion

In [15]:
stacked_sales <-
  sales %>%
  gather(key = auto_type,value = num_sales,-Salesperson)
head(stacked_sales)

Salesperson,auto_type,num_sales
Ann,Compact,22
Bob,Compact,19
Yolanda,Compact,19
Xerxes,Compact,12
Ann,Sedan,18
Bob,Sedan,12


### `spread()`

#### Imperative:

```{r}
spread(data, key = , value = )
```
#### Piping:

```{r}
data %>% 
  spread(key = , value = )
```

Arguments:

1. Column to split on
2. Column to split

#### A simple unstack

In [16]:
head(stacked_sales)

Salesperson,auto_type,num_sales
Ann,Compact,22
Bob,Compact,19
Yolanda,Compact,19
Xerxes,Compact,12
Ann,Sedan,18
Bob,Sedan,12


In [17]:
stacked_sales %>%
  spread(key = auto_type,value = num_sales)

Salesperson,Compact,Sedan,SUV,Truck
Ann,22,18,15,12
Bob,19,12,17,20
Xerxes,12,23,18,9
Yolanda,19,8,32,15


#### A stack + mutate + aggregate +  unstack

In [18]:
sales %>%
  gather(key = auto_type,value = num_sales,Compact:Truck) %>% 
  mutate(car_type = recode(auto_type,
                           'Compact' = 'car',
                           'Sedan' = 'car',
                           'SUV' = 'utility',
                           'Truck' = 'utility')) %>% 
   group_by(Salesperson, car_type) %>% 
   summarize(total_sales = sum(num_sales)) %>% 
   spread(key = car_type, value = total_sales)

`summarise()` regrouping output by 'Salesperson' (override with `.groups` argument)


Salesperson,car,utility
Ann,40,27
Bob,31,37
Xerxes,35,27
Yolanda,27,47


An alternative recode:

In [19]:
sales %>%
  gather(key = auto_type,value = num_sales,Compact:Truck) %>% 
  mutate(car_type = ifelse(auto_type %in% c('Compact','Sedan'), 
                           'car',
                           'utility')) %>% 
   group_by(Salesperson, car_type) %>% 
   summarize(total_sales = sum(num_sales)) %>% 
   spread(key = car_type, value = total_sales)

`summarise()` regrouping output by 'Salesperson' (override with `.groups` argument)


Salesperson,car,utility
Ann,40,27
Bob,31,37
Xerxes,35,27
Yolanda,27,47


# Joining 

In [20]:
emp <- read.csv('https://github.com/WSU-DataScience/DSCI_210_R_notebooks/raw/main/employee.csv')
dept <- read.csv('https://github.com/WSU-DataScience/DSCI_210_R_notebooks/raw/main/department.csv')
emp
dept

Last.Name,Department.ID
Rafferty,31
Jones,33
Heisenberg,33
Robinson,34
Smith,34
Williams,NA


Department.ID,Department.Name
31,Sales
33,Engineering
34,Clerical
35,Marketing


#### Inner join

In [21]:
inner_join(emp,dept,by='Department.ID')

Last.Name,Department.ID,Department.Name
Rafferty,31,Sales
Jones,33,Engineering
Heisenberg,33,Engineering
Robinson,34,Clerical
Smith,34,Clerical


#### Left join

In [22]:
left_join(emp,dept,by='Department.ID')

Last.Name,Department.ID,Department.Name
Rafferty,31,Sales
Jones,33,Engineering
Heisenberg,33,Engineering
Robinson,34,Clerical
Smith,34,Clerical
Williams,NA,NA


#### Right join

In [23]:
right_join(emp,dept,by='Department.ID')

Last.Name,Department.ID,Department.Name
Rafferty,31,Sales
Jones,33,Engineering
Heisenberg,33,Engineering
Robinson,34,Clerical
Smith,34,Clerical
NA,35,Marketing


# Translating JMP to `dplyr/tidyr`

#### JMP

* Visual
* Point-and-click
* No name or syntax errors
  * Still have semantic errors

#### `dplyr/tidyr`

* automated
* reproducible
* easy to make changes

### Implications for Translating dplyr to JMP

* Same actions, different syntax
* Use what you already know
    * Figure out process in JMP
    * Translate to `dplyr`

### Simple Aggregations

```{r}
surveys %>%
  summarize(
    avg_wgt = mean(weight, 
                   na.rm = TRUE))
```

<img width="850" src="https://raw.githubusercontent.com/WSU-DataScience/DSCI_210_R_notebooks/main/img/simple_aggregation_1.png">

<img width="850" src="https://raw.githubusercontent.com/WSU-DataScience/DSCI_210_R_notebooks/main/img/simple_aggregation_2.png">

### Group and Aggregate

```{r}
grp_agg <- 
  surveys %>%
    group_by(taxa) %>%
    summarize(avg = mean(weight, na.rm = TRUE))
grp_agg
```

<img width="850" src="https://raw.githubusercontent.com/WSU-DataScience/DSCI_210_R_notebooks/main/img/group_and_aggregate_0.png">

<img width="850" src="https://raw.githubusercontent.com/WSU-DataScience/DSCI_210_R_notebooks/main/img/group_and_aggregate_1.png">

<img width="850" src="https://raw.githubusercontent.com/WSU-DataScience/DSCI_210_R_notebooks/main/img/group_and_aggregate_2.png">

### Multiple Group Variables

<img width="850" src="https://raw.githubusercontent.com/WSU-DataScience/DSCI_210_R_notebooks/main/img/grp2_agg.PNG">

### Numeric transformations

<img width="850" src="https://raw.githubusercontent.com/WSU-DataScience/DSCI_210_R_notebooks/main/img/numeric_transform_0.png">

<img width="850" src="https://raw.githubusercontent.com/WSU-DataScience/DSCI_210_R_notebooks/main/img/numeric_transform_1.png">

<img width="850" src="https://raw.githubusercontent.com/WSU-DataScience/DSCI_210_R_notebooks/main/img/numeric_transform_2.png">

### Character transformations 

```{r}
stacked_sales
stacked_sales %>%
  mutate(car_type = 
           recode(auto_type, 
                  Compact = "Car",
                  Sedan = "Car",
                  SUV = "Utility",
                  Truck = "Utility"))
```

<img width="850" src="https://raw.githubusercontent.com/WSU-DataScience/DSCI_210_R_notebooks/main/img/character_transform_1.png">

<img width="850" src="https://raw.githubusercontent.com/WSU-DataScience/DSCI_210_R_notebooks/main/img/character_transform_2.png">

<img width="850" src="https://raw.githubusercontent.com/WSU-DataScience/DSCI_210_R_notebooks/main/img/character_transform_3.png">

<img width="850" src="https://raw.githubusercontent.com/WSU-DataScience/DSCI_210_R_notebooks/main/img/character_transform_4.png">


  

### Stack/`gather`

<img width="850" src="https://raw.githubusercontent.com/WSU-DataScience/DSCI_210_R_notebooks/main/img/translate_gather_0.png">

<img width="850" src="https://raw.githubusercontent.com/WSU-DataScience/DSCI_210_R_notebooks/main/img/translate_gather_1.png">

<img width="850" src="https://raw.githubusercontent.com/WSU-DataScience/DSCI_210_R_notebooks/main/img/translate_gather_2.png">

<img width="850" src="https://raw.githubusercontent.com/WSU-DataScience/DSCI_210_R_notebooks/main/img/translate_gather_3.png">

###  Row filter

```{r}
stacked_sales %>%
  filter(num_sales > 20)
```

<img width="850" src="https://raw.githubusercontent.com/WSU-DataScience/DSCI_210_R_notebooks/main/img/filter_update.png">

### Unstack/`spread()`

<img width="850" src="https://raw.githubusercontent.com/WSU-DataScience/DSCI_210_R_notebooks/main/img/unstack_1.png">

<img width="850" src="https://raw.githubusercontent.com/WSU-DataScience/DSCI_210_R_notebooks/main/img/unstack_2.png">

<img width="850" src="https://raw.githubusercontent.com/WSU-DataScience/DSCI_210_R_notebooks/main/img/unstack_3.png">